In [12]:
import numpy as np
import os
import matplotlib.pyplot as plt
import h5py
from scipy import interpolate
from ccfj import GetStationPairs
from ccfj import GetStationPairs
import pandas as pd
import yaml
import geopandas as gp
from geopy.distance import great_circle
import openpyxl
from openpyxl import load_workbook
from concurrent.futures import ThreadPoolExecutor

import time

In [13]:
import sys
sys.path.append(r'../tools_F-J/')
from toollib_standard import maplib
from toollib_standard import mathlib
from toollib_standard import filelib
from toollib_standard import stacklib
from toollib_standard import plotlib

In [14]:
with open('a-project_repar.yml', 'r', encoding='utf-8') as f:
    proj = yaml.load(f.read(), Loader=yaml.FullLoader)
name_project = proj['name']
#name_project = 'project_repartrition/repartrition_01-03/'               # Harmon server
name_project

'project_repartition_v3.0/output_repar_v9.1_test/'

In [15]:
with open('0_config.yml', 'r', encoding='utf-8') as f:
    dir_config = yaml.load(f.read(), Loader=yaml.FullLoader)
dir_project_workspace = dir_config['dir_project_workspace']
dir_CC_workspace = dir_config['dir_CC_workspace']
print('dir_CC_workspace: ', dir_CC_workspace)
print('dir_project_workspace: ', dir_project_workspace)
dir_project = os.path.join(dir_project_workspace, name_project)
print('dir_project: ', dir_project)

dir_CC_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project_workspace:  /shdisk/rem2/Harmon/F-J/San/
dir_project:  /shdisk/rem2/Harmon/F-J/San/project_repartition_v3.0/output_repar_v9.1_test/


In [16]:
filename = dir_project+'Basic_info.yml'
with open(filename, 'r', encoding='utf-8') as f:
    info_basic = yaml.load(f.read(), Loader=yaml.FullLoader)
filename_bi = dir_project+'Basic_info.npy'
info_basic_bi = np.load(filename_bi, allow_pickle='TRUE').item()      # setting dictionary

In [17]:
key_subworks = info_basic['key_subworks']
key_subworks

['1--08-03',
 '2--09-03',
 '3--10-03',
 '4--11-03',
 '5--12-03',
 '6--13-03',
 '7--14-03',
 '8--15-03',
 '9--16-03',
 '10--18-03',
 '11--19-03',
 '12--20-03',
 '13--21-03',
 '14--22-03',
 '15--23-03',
 '16--24-03',
 '17--25-03',
 '18--26-03']

In [18]:
dir_stack = dir_project+info_basic['dir_stack']
dir_CC = dir_CC_workspace+info_basic['name_CC']
dir_partition = dir_project + info_basic['dir_partition']
#dir_CC = dir_CC_workspace+'CC/CC_40_prewhiten/'

In [19]:
stalistname_all = info_basic['stalistname_all']
stainfo = pd.read_excel(stalistname_all)
nsta_all = len(stainfo.iloc[:,0])
StationPairs_all = GetStationPairs(nsta_all)
nPairs_all = int(len(StationPairs_all)/2)
stalist_all = stainfo['Station'].tolist()
lat_all = stainfo['latitude'].tolist() 
lon_all = stainfo['longitude'].tolist()

In [20]:
ncffile = h5py.File( dir_CC+ 'gather_all.h5','r')
ncfs = ncffile['ncfs'][:]
f = ncffile['f'][:]
count_all = ncffile['count'][:]
ncffile.close()

In [21]:
info_basic_bi['r_max'] = {}

In [22]:
def linear_stack(key_subwork):
    global key_subworks
    global dir_stack
    global info_basic
    global stainfo_all
    global dir_partition

    print("Collecting ",key_subwork,' ...')
    if os.path.exists(dir_stack+str(key_subwork)+'_gather_linear.h5'):
        print("File exists.")
        return

    time0 = time.time()

    nf = info_basic['nf']
    
    filepath = dir_partition + str(key_subwork) + '.txt'
    stalist, lat, lon = np.loadtxt(filepath, dtype='str', unpack=True)
    nsta = len(stalist)
    StationPairs = GetStationPairs(nsta)
    nPairs = int(len(StationPairs)/2)

    ncfs_sum_linear = np.zeros((nPairs,nf),dtype=np.complex64)
    r = np.zeros(nPairs)
    count= np.zeros(nPairs)
    StationPairs = GetStationPairs(nsta)
    for i in range(nPairs):
        sta1 = StationPairs[2*i]
        sta2 = StationPairs[2*i+1]
        idx1 = np.min( [int(stalist_all.index(stalist[sta1])),int(stalist_all.index(stalist[sta2]))] )
        idx2 = np.max( [int(stalist_all.index(stalist[sta1])),int(stalist_all.index(stalist[sta2]))] )
        #idx1 = int(stalist_all.index(stalist[sta1]))
        #idx2 = int(stalist_all.index(stalist[sta2]))
        #idx1 = int(stainfo[stainfo[key_pd]==stalist[sta1]].index.values[0])
        #idx2 = int(stainfo[stainfo[key_pd]==stalist[sta2]].index.values[0])
        
        m = 0
        for j in range(nsta_all-idx1,nsta_all):
            m += j
        num = m +idx2 - idx1 -1
        
        ncfs_sum_linear[i,:] = np.nan_to_num(ncfs[num,:])
        count[i] = count_all[num]
        #r[i] = r0[num]
        #if count_all[num] > 0:
        #    ncfs_sum_linear[i,:] = ncfs[num,:]/count_all[num]
        #    count[i] = count_all[num]

        r[i] = great_circle((lat[sta1],lon[sta1]),(lat[sta2],lon[sta2])).km
        #names.append([stalist_all[StationPairs_all[2*num]],stalist_all[StationPairs_all[2*num+1]]])

    outname = str(key_subwork)+'_gather_linear.h5'
    if os.path.exists(dir_stack+outname):
        os.remove(dir_stack+outname)
    ncffile = h5py.File(dir_stack+outname,'w')
    
    ncffile.create_dataset('ncfs',data=ncfs_sum_linear)
    ncffile.create_dataset('r',data=r)
    ncffile.create_dataset('count',data=count)
    ncffile.create_dataset('f',data=f)
    ncffile.create_dataset('StationPairs',data=StationPairs)
    #print(ncffile.keys())
    ncffile.close()
    #np.savez(dir_stack+key_subwork+"_summed-linear.npz",ncfs= ncfs_sum_linear,r = r,stalist=stalist,StationPairs=StationPairs)
    print("Done in ",time.time()-time0," s.", str(key_subworks.index(key_subwork)+1),'/',len(key_subworks))

In [23]:
nThreads = 60

In [24]:
for key_subwork in key_subworks:
    linear_stack(key_subwork)

Done in  0.234389066696167  s. 1 / 18
Done in  0.24254512786865234  s. 2 / 18
Done in  0.5593142509460449  s. 3 / 18
Done in  0.44658613204956055  s. 4 / 18
Done in  0.41878652572631836  s. 5 / 18
Done in  0.4696981906890869  s. 6 / 18
Done in  0.45261216163635254  s. 7 / 18
Done in  0.47664666175842285  s. 8 / 18
Done in  0.5137462615966797  s. 9 / 18
Done in  0.606677770614624  s. 10 / 18
Done in  0.738044261932373  s. 11 / 18
Done in  0.7895646095275879  s. 12 / 18
Done in  0.684561014175415  s. 13 / 18
Done in  0.432894229888916  s. 14 / 18
Done in  0.2493281364440918  s. 15 / 18
Done in  0.4061882495880127  s. 16 / 18
Done in  0.5876445770263672  s. 17 / 18
Done in  0.5517878532409668  s. 18 / 18


In [25]:
np.save(filename_bi,info_basic_bi)